In [58]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import glob
import xarray as xr
from matplotlib import pyplot as plt
%matplotlib qt
import cartopy.crs as ccrs
import numpy as np
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
#import dictionaries from my_tools to automize plotting
from my_tools import plot_dic
from mpl_toolkits.basemap import Basemap, addcyclic
import matplotlib
matplotlib.use('agg')
#Set matplotlib to use agg so the plot doesn't constantly show up when I produce it which is very annoying.




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Normal 10 lead day error plot

In [60]:
#list all the relevant files
#Set variable to plot 
variablez = ['envelope','phasespeed','u_wind']
path_file = '/media/onno/Algemeen/Thesis/'
path = '/media/onno/Algemeen/Thesis/GFS_mean/'
vminmax = [(-2,2),(-2,2),(-3,3)]
for j,variable in enumerate(variablez):
    if variable!='phasespeed':
        continue
    filez = sorted(glob.glob(path+'/*{}_day_??_globe_mars.nc'.format(plot_dic[variable]['filename'])))
    #create orthographic plot for each lead day
    latz = [(85,0),(0,-85)]
    for lat_hem in latz:
        fig,axz=plt.subplots(2,5,figsize=(20,10))

        #Looping over all lead day error files
        for i, ax in enumerate(axz.flat):
            ds = xr.open_dataset(filez[i+1],decode_times=False).squeeze()
            #Discard 85 - 90 degrees north, because these data yield unrealistical large errors
            ds = ds.sel(lat=slice(lat_hem[0],lat_hem[1]))
            ds_cyclic, lon_cyclic = addcyclic(ds[plot_dic[variable]['data_variable']].values, ds.lon)
            lon,lat = np.meshgrid(lon_cyclic,ds.lat)
            #Create coastlines
            if lat_hem == (85,0):
                m = Basemap(projection='ortho',lon_0=0,lat_0=90,resolution='c',ax=ax)
            else:
                m = Basemap(projection='ortho',lon_0=0,lat_0=-90,resolution='c',ax=ax)
            #Plotting the data
            #Sometimes contourf function raises unexplainable ValueError (might be bug). Then I switch to pcolormesh
            im = m.contourf(lon,lat,ds_cyclic,latlon='true', extend='both', cmap=plt.cm.get_cmap('bwr'),
               vmin=vminmax[j][0],vmax=vminmax[j][1],levels=np.linspace(vminmax[j][0],vminmax[j][1],49))
            m.drawcoastlines()
            m.drawparallels((-80,-60,-40,-20,0,20,40,60,80))
            m.drawmeridians((0,60,120,180,240,300),labels=[0,0,1,1],labelstyle='+/-')
            #Plotting the variable mean error below the graph
            #Only taking data between 80 and 30 degrees N because other data is not representative for mid-latitudes
            #For Souther Hemisphere we consider latitudes between -30 and -80 degrees N
            if lat_hem==(85,0):
                ds_text = ds.sel(lat=slice(80,30))
            else:
                ds_text = ds.sel(lat=slice(-30,-80))
            ax.text(x=0.5,y=-0.125,s='Mean error = {:.2f}'.format(float(my_tools.weighted_average_area_2D(ds_text))),
                   transform=ax.transAxes,horizontalalignment='center',fontsize=12)
            #set gridlines
            ax.text(x=0.5,y=1.10,s='Day {}'.format(i+1),transform=ax.transAxes,horizontalalignment='center',fontsize=12)

        #set figure size
        fig.suptitle('GEFS Mean Annual {} Error Compared to ERA5'.format(plot_dic[variable]['title']))
        #Make room for color bar and create color bar
        fig.subplots_adjust(left=0.03,right=0.97,bottom=0.135,top=0.9)
        cbax = fig.add_axes([0.35, 0.06, 0.3, 0.025])
        fig.colorbar(im, cax=cbax,ticks=np.linspace(vminmax[j][0],vminmax[j][1],5),orientation='horizontal')
        cbax.set_xlabel(plot_dic[variable]['label'],rotation=0)
        if lat_hem==(85,0): 
            fig.savefig(path+'/GEFS_forecast_mean_{}_error_NH.png'.format(plot_dic[variable]['filename']))
        else:
            fig.savefig(path+'/GEFS_forecast_mean_{}_error_SH.png'.format(plot_dic[variable]['filename']))
        plt.close(fig)

Simple 1 lead day error plot


In [5]:
#list all files
variables = ['envelope']
latz_hem = [(85,0),(0,-85)]

#Loop over all files
for i, variable in enumerate(variables):
    for lat_hem in latz_hem:
        #open associated file for lead day
        if lat_hem == (85,0):
            lat_0 = 90
        else:
            lat_0 = -90
        file = path+'GFS_mean_error_{}_day_00_globe_mars.nc'.format(variable)
        ds = xr.open_dataset(file,decode_times=False).squeeze()
        ds = ds.sel(lat=slice(lat_hem[0],lat_hem[1]))
        #create figure
        fig,ax = plt.subplots()

        ds_cyclic, lon_cyclic = addcyclic(ds[plot_dic[variable]['data_variable']].values, ds.lon)
        lon,lat = np.meshgrid(lon_cyclic,ds.lat)

        m = Basemap(projection='ortho',lon_0=0,lat_0=lat_0,resolution='c',ax=ax)
        im = m.contourf(lon,lat,ds_cyclic,latlon='true', extend='both', cmap='bwr',
                   vmin=-1,vmax=1,levels=np.linspace(-1,1,41))
        m.drawcoastlines()
        m.drawparallels((-80,-60,-40,-20,0,20,40,60,80))
        m.drawmeridians((0,60,120,180,240,300),labels=[0,0,1,1])
        #create colorbar
        cbar = fig.colorbar(im)
        cbar.set_label(plot_dic[variable]['label'], rotation=90)
        fig.suptitle('GEFS mean RWP {} error day {}'.format(plot_dic[variable]['title'],i))
#         if lat_hem == (85,0):
#             fig.savefig(path+'GFS_mean_error_{}_initialization_NH.png'.format(variable))
#         else:
#             fig.savefig(path+'GFS_mean_error_{}_initialization_SH.png'.format(variable))
#         plt.close(fig)
# #     fig.savefig('/media/onno/Algemeen/Thesis/GFS_mean/GEFS_forecast_{}_error_day_{:02d}'.format(plot_dic[variable]['filename'],i))


/home/onno/miniconda3/envs/thesis/lib/python3.8/site-packages/mpl_toolkits/basemap/__init__.py:5098: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)
/home/onno/miniconda3/envs/thesis/lib/python3.8/site-packages/mpl_toolkits/basemap/__init__.py:5098: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return npsel.concatenate((a,a[slicer]),axis=axis)


Normal 10 day plot for each season

In [61]:
#Loop over all seasons
path = '/media/onno/Algemeen/Thesis/GFS_mean_season/'
variablez = ['envelope','phasespeed','u_wind']
seasons = ['MAM','JJA','SON','DJF']
vminmax = [(-2,2),(-2,2),(-3,3)]
for j,variable in enumerate(variablez):
    if variable!='phasespeed':
        continue
    for season in seasons:
        latz = [(85,0),(0,-85)]
        for lat_hem in latz:
            #list all files for each season
            filez = sorted(glob.glob(path+'/*{}_{}*globe.nc'.format(plot_dic[variable]['filename'],season)))
            #create orthographic plot for each lead day
            fig,axz = plt.subplots(2,5,figsize=(20,10))
            #Plot each mean error for each lead day
            for i, ax in enumerate(axz.flat):
                #open accompanying netcdf file for lead day error
                ds = xr.open_dataset(filez[i+1],decode_times=False).squeeze()
                #Discard 85 - 90 degrees north, because these data yield unrealistical large errors
                ds = ds.sel(lat=slice(lat_hem[0],lat_hem[1]))
                ds_cyclic, lon_cyclic = addcyclic(ds[plot_dic[variable]['data_variable']].values, ds.lon)
                lon,lat = np.meshgrid(lon_cyclic,ds.lat)
                if lat_hem == (85,0):
                    m = Basemap(projection='ortho',lon_0=0,lat_0=90,resolution='c',ax=ax)
                else:
                    m = Basemap(projection='ortho',lon_0=0,lat_0=-90,resolution='c',ax=ax)
                #Plotting the data
                #Sometimes contourf function raises unexplainable ValueError (might be bug). Then I switch to pcolormesh
                im = m.contourf(lon,lat,ds_cyclic,latlon='true', extend='both', cmap=plt.cm.get_cmap('bwr'),
                   vmin=vminmax[j][0],vmax=vminmax[j][1],levels=np.linspace(vminmax[j][0],vminmax[j][1],49))
                m.drawcoastlines()
                m.drawparallels((-80,-60,-40,-20,0,20,40,60,80))
                m.drawmeridians((0,60,120,180,240,300),labels=[0,0,1,1],labelstyle='+/-')
                if lat_hem==(85,0):
                    ds_text = ds.sel(lat=slice(80,30))
                else:
                    ds_text = ds.sel(lat=slice(-30,-80))
                ax.text(x=0.5,y=-0.125,s='Mean error = {:.2f}'.format(float(my_tools.weighted_average_area_2D(ds_text))),
                       transform=ax.transAxes,horizontalalignment='center',fontsize=12)
                #set gridlines
                ax.text(x=0.5,y=1.10,s='Day {}'.format(i+1),transform=ax.transAxes,horizontalalignment='center',fontsize=12)

            # cbar = fig.colorbar(im)
            # cbar.set_label('E (m/s)', rotation=90)
            fig.suptitle('GEFS Seasonal Mean {} Error {} Compared to ERA5'.format(plot_dic[variable]['title'],season))
            #Set colorbar and make room for it
            fig.subplots_adjust(left=0.03,right=0.97,bottom=0.135,top=0.9)
            cbax = fig.add_axes([0.35, 0.06, 0.3, 0.025])
            fig.colorbar(im, cax=cbax,ticks=np.linspace(vminmax[j][0],vminmax[j][1],5),orientation='horizontal')
            cbax.set_xlabel(plot_dic[variable]['label'],rotation=0)
            #Save figure and make sure it does not open a window
            if lat_hem == (85,0):
                fig.savefig(path+'/GEFS_forecast_{}_error_{}_NH.png'.format(plot_dic[variable]['filename'],season),dpi=300)
            else:
                fig.savefig(path+'/GEFS_forecast_{}_error_{}_SH.png'.format(plot_dic[variable]['filename'],season),dpi=300)
            plt.close(fig)